<a href="https://colab.research.google.com/github/BR49/RAG-for-Summarization/blob/main/RAG_for_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faiss-cpu sentence-transformers transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
import zipfile
import requests
import os
import io

# Download the dataset
url = "https://static.nhtsa.gov/odi/ffdd/rcl/FLAT_RCL.zip"
zip_path = "FLAT_RCL.zip"

if not os.path.exists(zip_path):
    response = requests.get(url, stream=True)  # Stream the response to avoid loading it all in memory
    response.raise_for_status()  # Check for bad responses (4xx or 5xx)
    with open(zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):  # Write in chunks
            f.write(chunk)

# Extract the dataset
# Instead of using BytesIO, try opening the zip file directly
# This is because the file was already correctly downloaded and saved to disk
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("recalls_data")

In [4]:
import pandas as pd

# readinag given csv file
# and creating dataframe
# Specify the delimiter as tab ('\t') to correctly parse the file
# Use the 'on_bad_lines='skip'' argument to skip rows with incorrect number of fields
df = pd.read_csv("recalls_data/FLAT_RCL.txt", delimiter='\t', on_bad_lines='skip')

# storing this dataframe in a csv file
df.to_csv('rcl.csv',
                  index = None)

<ipython-input-4-b70e831afa70>:7: DtypeWarning: Columns (17,18,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("recalls_data/FLAT_RCL.txt", delimiter='\t', on_bad_lines='skip')


In [5]:
import pandas as pd

df = pd.read_csv("recalls_data/FLAT_RCL.txt", delimiter='\t', on_bad_lines='skip', header=None)  # Add header=None
#df.columns = df.iloc[0]  # Set the first row as column names
#df = df[1:]  # Remove the original first row (now used as column names)
df.to_csv('rcl.csv', index=None)

<ipython-input-5-39feb9f5e6df>:3: DtypeWarning: Columns (17,18,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("recalls_data/FLAT_RCL.txt", delimiter='\t', on_bad_lines='skip', header=None)  # Add header=None


In [5]:
print(df.iloc[:, 0])

0              1
1              2
2              3
3              4
4              5
           ...  
268959    268963
268960    268964
268961    268965
268962    268966
268963    268967
Name: 0, Length: 268964, dtype: int64


In [6]:
num_cols = df.shape[1]
df.columns = [f'Column{i+1}' for i in range(num_cols)] # Dynamically create column names

print(df)


        Column1    Column2          Column3         Column4  Column5 Column6  \
0             1  02V288000             FORD           FOCUS     2000   02S41   
1             2  02V288000             FORD           FOCUS     2001   02S41   
2             3  02V236000            JAYCO  FT EAGLE 10 SG     2003     NaN   
3             4  02V237000  HOLIDAY RAMBLER        ENDEAVOR     2000     NaN   
4             5  02V237000  HOLIDAY RAMBLER        ENDEAVOR     1999     NaN   
...         ...        ...              ...             ...      ...     ...   
268959   268963  24V763000            HONDA     CR-V HYBRID     2024     PJW   
268960   268964  24V763000            HONDA     CR-V HYBRID     2024     PJW   
268961   268965  24V763000            HONDA     CR-V HYBRID     2024     PJW   
268962   268966  24V763000            HONDA     CR-V HYBRID     2024     PJW   
268963   268967  24V763000            HONDA     CR-V HYBRID     2024     PJW   

                                       

In [7]:
df = df.rename(columns={'Column3': 'Make'})
print(df['Make'])
df.to_csv('rcl_processed.csv', index=False)

0                    FORD
1                    FORD
2                   JAYCO
3         HOLIDAY RAMBLER
4         HOLIDAY RAMBLER
               ...       
268959              HONDA
268960              HONDA
268961              HONDA
268962              HONDA
268963              HONDA
Name: Make, Length: 268964, dtype: object


In [8]:
df = df.rename(columns={'Column4': 'Model'})
print(df['Model'])
df.to_csv('rcl_processed.csv', index=False)

0                  FOCUS
1                  FOCUS
2         FT EAGLE 10 SG
3               ENDEAVOR
4               ENDEAVOR
               ...      
268959       CR-V HYBRID
268960       CR-V HYBRID
268961       CR-V HYBRID
268962       CR-V HYBRID
268963       CR-V HYBRID
Name: Model, Length: 268964, dtype: object


In [9]:
df = df.rename(columns={'Column5': 'Year'})
print(df['Year'])
df.to_csv('rcl_processed.csv', index=False)


0         2000
1         2001
2         2003
3         2000
4         1999
          ... 
268959    2024
268960    2024
268961    2024
268962    2024
268963    2024
Name: Year, Length: 268964, dtype: int64


In [10]:
print(df['Column20'])

0         CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
1         CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
2         ON CERTAIN FOLDING TENT CAMPERS, THE FEDERAL C...
3         ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...
4         ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...
                                ...                        
268959    Honda (American Honda Motor Co.) is recalling ...
268960    Honda (American Honda Motor Co.) is recalling ...
268961    Honda (American Honda Motor Co.) is recalling ...
268962    Honda (American Honda Motor Co.) is recalling ...
268963    Honda (American Honda Motor Co.) is recalling ...
Name: Column20, Length: 268964, dtype: object


In [10]:
df = df.rename(columns={'Column20': 'Defect Summary'})
print(df['Defect Summary'])
df.to_csv('rcl_processed.csv', index=False)

0         CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
1         CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
2         ON CERTAIN FOLDING TENT CAMPERS, THE FEDERAL C...
3         ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...
4         ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...
                                ...                        
268959    Honda (American Honda Motor Co.) is recalling ...
268960    Honda (American Honda Motor Co.) is recalling ...
268961    Honda (American Honda Motor Co.) is recalling ...
268962    Honda (American Honda Motor Co.) is recalling ...
268963    Honda (American Honda Motor Co.) is recalling ...
Name: Defect Summary, Length: 268964, dtype: object


In [11]:
df = df.rename(columns={'Column21': 'Consequence Summary'})
print(df['Consequence Summary'])
df.to_csv('rcl_processed.csv', index=False)

0         THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...
1         THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...
2         IF THE TIRES WERE INFLATED TO 80 PSI, THEY COU...
3         CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...
4         CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...
                                ...                        
268959    A fuel leak in the presence of an ignition sou...
268960    A fuel leak in the presence of an ignition sou...
268961    A fuel leak in the presence of an ignition sou...
268962    A fuel leak in the presence of an ignition sou...
268963    A fuel leak in the presence of an ignition sou...
Name: Consequence Summary, Length: 268964, dtype: object


In [12]:
df = df.rename(columns={'Column22': 'Corrective Summary'})
print(df['Corrective Summary'])
df.to_csv('rcl_processed.csv', index=False)

0         DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
1         DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...
2         OWNERS WILL BE MAILED CORRECT LABELS FOR INSTA...
3         DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...
4         DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...
                                ...                        
268959    Dealers will inspect and replace the high-pres...
268960    Dealers will inspect and replace the high-pres...
268961    Dealers will inspect and replace the high-pres...
268962    Dealers will inspect and replace the high-pres...
268963    Dealers will inspect and replace the high-pres...
Name: Corrective Summary, Length: 268964, dtype: object


In [14]:
print(df)

        Column1    Column2             Make           Model  Year Column6  \
0             1  02V288000             FORD           FOCUS  2000   02S41   
1             2  02V288000             FORD           FOCUS  2001   02S41   
2             3  02V236000            JAYCO  FT EAGLE 10 SG  2003     NaN   
3             4  02V237000  HOLIDAY RAMBLER        ENDEAVOR  2000     NaN   
4             5  02V237000  HOLIDAY RAMBLER        ENDEAVOR  1999     NaN   
...         ...        ...              ...             ...   ...     ...   
268959   268963  24V763000            HONDA     CR-V HYBRID  2024     PJW   
268960   268964  24V763000            HONDA     CR-V HYBRID  2024     PJW   
268961   268965  24V763000            HONDA     CR-V HYBRID  2024     PJW   
268962   268966  24V763000            HONDA     CR-V HYBRID  2024     PJW   
268963   268967  24V763000            HONDA     CR-V HYBRID  2024     PJW   

                                             Column7  \
0       ELECTRICAL 

In [13]:
filtered_df = df[df['Make'].str.lower().isin(['ford', 'toyota'])]
filtered_df = filtered_df[['Make', 'Model', 'Year', 'Defect Summary', 'Consequence Summary', 'Corrective Summary']]

# Combine summaries into a single text
filtered_df['Document'] = (
    filtered_df['Defect Summary'].fillna('') + ' ' +
    filtered_df['Consequence Summary'].fillna('') + ' ' +
    filtered_df['Corrective Summary'].fillna('')
)

# Save the filtered dataset for quick access
filtered_df.to_csv("filtered_recalls.csv", index=False)

In [17]:
print(filtered_df)

        Make           Model  Year  \
0       FORD           FOCUS  2000   
1       FORD           FOCUS  2001   
148     FORD  CROWN VICTORIA  2002   
300     FORD       NAVIGATOR  2000   
301     FORD      EXPEDITION  2000   
...      ...             ...   ...   
268321  FORD         MUSTANG  2024   
268868  FORD         TRANSIT  2023   
268869  FORD         TRANSIT  2023   
268870  FORD         TRANSIT  2024   
268871  FORD         TRANSIT  2024   

                                           Defect Summary  \
0       CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...   
1       CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...   
148     ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT...   
300     CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...   
301     CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...   
...                                                   ...   
268321  Ford Motor Company (Ford) is recalling certain...   
268868  Ford Motor Company (Ford) is recalling certain.

In [14]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load the dataset
data = pd.read_csv("filtered_recalls.csv")

# Load an embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all documents
embeddings = embedding_model.encode(data['Document'].tolist(), show_progress_bar=True)

# Create a FAISS index for retrieval
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Save the index for later use
faiss.write_index(index, "recalls_index.faiss")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/535 [00:00<?, ?it/s]

In [15]:
def retrieve_documents(query, top_k=5):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    retrieved_docs = data.iloc[indices[0]]['Document'].tolist()
    return retrieved_docs

from sklearn.metrics.pairwise import cosine_similarity

def filter_relevant_docs(query_embedding, doc_embeddings, docs, threshold=0.4):
    """
    Filters documents based on similarity to the query embedding.
    """
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    filtered_docs = [docs[i] for i, sim in enumerate(similarities) if sim > threshold]
    return filtered_docs


In [ ]:
!pip install transformers

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `BharathR` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `BharathR`


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B")

# Set pad_token_id to eos_token_id
tokenizer.pad_token_id = tokenizer.eos_token_id

def summarize_text(docs):
    unique_docs = list(set(docs))
    input_text = " ".join(unique_docs)
    #input_text = " ".join(docs)
    inputs = tokenizer(input_text, return_tensors="pt", max_length=150, truncation=True, padding=True) # Add padding=True

    # Pass the attention mask to model.generate
    summary_ids = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,  # Pass attention mask
        max_new_tokens=150,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [19]:
def rag_agent_with_relevance_check(input_json):
    # Step 1: Parse input and create query
    make = input_json.get("make", "")
    model = input_json.get("model", "")
    year = input_json.get("year", "")
    issue = input_json.get("issue", "")
    query = f"{make} {model} {year} {issue}".strip()

    # Step 2: Retrieve relevant documents
    query_embedding = embedding_model.encode([query])[0]
    docs = retrieve_documents(query)
    doc_embeddings = embedding_model.encode(docs)

    # Step 3: Filter for relevance
    filtered_docs = filter_relevant_docs(query_embedding, doc_embeddings, docs)

    # Step 4: Check if any documents are relevant
    if not filtered_docs:
        return {"summary": "No relevant recalls found for the specified input.", "retrieved_documents": []}

    # Step 5: Summarize the relevant documents
    summary = summarize_text(filtered_docs)
    return {"summary": summary, "retrieved_documents": filtered_docs}


input_example = {#"make": "unknownmake", "model": "unknownmodel", "year": 2025, "issue": "unknown issue"}
    'make': 'ford',
    'model': 'escape',
    'year': '2001',
    'issue': 'stuck throttle risk'
}
# Run the RAG agent
result = rag_agent_with_relevance_check(input_example)
print("Summary:", result['summary'])
print("Retrieved Documents:",result['retrieved_documents'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: Ford Motor Company is recalling certain model year 2001 through 2004 Escape vehicles equipped with 3.0L V6 engines and speed control manufactured from October 22, 1999, through January 23, 2004.  Inadequate clearance between the engine cover and the speed control cable connector could result in a stuck throttle when the accelerator pedal is fully or almost-fully depressed.  This risk exists regardless of whether or not speed control (cruise control) is used. A stuck throttle may result in very high vehicle speeds and make it difficult to stop or slow the vehicle, which could cause a crash, serious injury or death.  Ford will notify owners, and dealers will repair the vehicles by increasing the engine cover clearance, free of charge.  The safety recall is expected to begin on, or about, September 13, 2010.  Owners may contact Ford Motor Company Customer Relationship Center at 1-866-436-7332.
Ford Motor Company is recalling certain model year 2001 through 2004 Escape vehicles eq

In [24]:
def test_rag_agent(test_cases):
    for i, case in enumerate(test_cases):
        print(f"\nTest Case {i+1}:")
        print(f"Input: {case}")
        result = rag_agent_with_relevance_check(case)
        print(f"Summary: {result['summary']}")
        print(f"Retrieved Documents: {result['retrieved_documents']}\n")

# Example test cases
test_cases = [
    {"make": "ford", "model": "escape", "year": 2005, "issue": "brake failure risk"},
    {"make": "toyota", "model": "camry", "year": 2012, "issue": "seatbelt defect"},
    {"make": "ford", "model": "focus", "year": 1990, "issue": "electric sunroof failure"},
    {"make": "toyota", "model": "", "year": "", "issue": "fuel system defect"},
    {"make": "ford", "model": "explorer", "year": 2003, "issue": "tire pressure sensor malfunction"},
    {"make": "toyota", "model": "", "year": "", "issue": "top recalls"},
    {"make": "toyota", "model": "rav4", "year": 2010, "issue": "engine overheating and power steering failure"},
    {"make": "", "model": "focus", "year": 2003, "issue": "engine stalling"},
    {"make": "unknownmake", "model": "unknownmodel", "year": 2025, "issue": "unknown issue"}
]

test_rag_agent(test_cases)


Test Case 1:
Input: {'make': 'ford', 'model': 'escape', 'year': 2005, 'issue': 'brake failure risk'}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: Ford Motor Company (Ford) is recalling certain 2018 Ford Escape and Lincoln MKC vehicles.  These vehicles may be equipped with incorrect front brake hoses that may chafe against other components, possibly resulting in a brake fluid leak. A brake fluid leak may lengthen the distance needed to stop the vehicle, increasing the risk of a crash. Ford will notify owners, and dealers will inspect the brake hoses, replacing any that are incorrect, free of charge.  The recall is expected to begin March 2018.  Owners may contact Ford customer service at 1-866-436-7332.  Ford's number for this recall is 18S04. Ford Motor Company (Ford) is recalling certain 2014-2016 Ford Fiesta, 2013-2016 Ford Fusion and 2015-2016 Ford Escape vehicles equipped with 1.6L GTDI engines. If the vehicle is started and driven with an insufficient level of coolant within the engine cooling system, the engine cylinder head may overheat, crack and leak oil. Ford will notify owners, and dealers will install a cool

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: Toyota Motor Engineering & Manufacturing (Toyota) is recalling certain 2022 Sienna vehicles equipped with seating for eight passengers.  An incorrect seat belt webbing guide may have been installed on the outboard second-row seat belt assemblies.   During seat belt pre-tensioner deployment, the incorrect webbing guide may damage the seat belt webbing, causing it to tear and increasing the risk of injury during a crash. Dealers will replace the outboard second-row seat belt assemblies, free of charge.  Owner notification letters were mailed January 10, 2022.  Owners may contact Toyota customer service at  1-800-331-4331.  Toyota's numbers for this recall are 21TB08 and 21TA08.
Toyota Motor Engineering & Manufacturing (Toyota) is recalling certain 2022 Sienna vehicles equipped with seating for eight passengers.  An incorrect seat belt webbing guide may have been installed on the outboard second-row seat belt assemblies.   During seat belt pre-tensioner deployment, the incorrect 

...

Summary: Toyota Motor Engineering & Manufacturing (Toyota) is recalling certain model year 2012-2015 Yaris vehicles manufactured August 31, 2011, to February 9, 2015, and sold or registered in Puerto Rico. The affected vehicles may have been manufactured with a roof headliner that does not provide the proper occupant protection in the event of a crash.  As such, these vehicles fail to comply with the requirements of Federal Motor Vehicle Safety Standard (FMVSS) No. 201, " Occupant Protection in Interior Impact." If the vehicles are equipped with a headliner that does not meet the impact requirements, there is an increased risk of occupantl injury in the event of a crash. Toyota will notify owners, and dealers will replace the headliner, free of charge. The manufacturer has not yet provided a notification schedule. Owners may contact Toyota customer service at 1-800-331-4331. Toyota's number for this recall is C0L.  Owners may also contact the National Highway Traffic Safety Administrat

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: Toyota Motor Engineering & Manufacturing (Toyota) is recalling certain 2019-2020 RAV4 and 2020 RAV4 Hybrid vehicles equipped with electric power steering (EPS).  Water may enter through the steering gear box cover and cause a loss of electric power steering assist.  A loss of power steering assist could increase steering effort at low vehicle speeds, increasing the risk of a crash. Toyota will notify owners, and dealers will replace the power steering gear box, free of charge.  The recall began August 17, 2020.  Owners may contact Toyota customer service at 1-888-270-9371.  Toyota's number for this recall is 20TB11 / 20TA11. 
Toyota Motor Engineering & Manufacturing (Toyota) is recalling certain 2019-2020 RAV4 and 2020 RAV4 Hybrid vehicles equipped with electric power steering (EPS).  Water may enter through the steering gear box cover and cause a loss of electric power steering assist.  A loss of power steering assist could increase steering effort at low vehicle speeds, incr

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: CERTAIN PICKUP TRUCKS, SPORT UTILITY VEHICLES AND VANS EQUIPPED WITH 6.0L DIESEL ENGINES MAY EXPERIENCE STALLING WITHOUT WARNING WHILE DRIVING AND MAY OR MAY NOT RESTART.  SHOULD THE ENGINE STALL, A VEHICLE CRASH COULD OCCUR. DEALERS WILL UPGRADE THE FUEL INJECTION CONTROL MODULE (FICM) WIRE HARNESS OR REPLACED, AND/OR HAVE A NEW INJECTION CONTROL PRESSURE (ICP)) SENSOR CONNECTOR INSTALLED FREE OF CHARGE.  THE RECALL BEGAN ON JULY 22, 2005.  OWNERS MAY CONTACT FORD AT 1-800-392-3673 OR LINCOLN AT 1-800-521-4140.
Retrieved Documents: ['CERTAIN PICKUP TRUCKS, SPORT UTILITY VEHICLES AND VANS EQUIPPED WITH 6.0L DIESEL ENGINES MAY EXPERIENCE STALLING WITHOUT WARNING WHILE DRIVING AND MAY OR MAY NOT RESTART.  SHOULD THE ENGINE STALL, A VEHICLE CRASH COULD OCCUR. DEALERS WILL UPGRADE THE FUEL INJECTION CONTROL MODULE (FICM) WIRE HARNESS OR REPLACED, AND/OR HAVE A NEW INJECTION CONTROL PRESSURE (ICP)) SENSOR CONNECTOR INSTALLED FREE OF CHARGE.  THE RECALL BEGAN ON JULY 22, 2005.  OWNE

In [20]:
!pip install rouge
from rouge import Rouge

def evaluate_summarization(generated_summaries, reference_summaries):
    """
    Evaluates the summarization component of the RAG pipeline using ROUGE.

    Parameters:
        generated_summaries (list): Summaries generated by the model.
        reference_summaries (list): Ground-truth reference summaries.

    Returns:
        dict: ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    rouge = Rouge()
    scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)
    return scores


In [21]:
def evaluate_pipeline(test_cases, true_relevance, reference_summaries):
    """
    Evaluates the RAG pipeline.

    Parameters:
        test_cases (list): List of test cases (query dictionaries).
        true_relevance (list of list): Ground-truth relevance for retrieval.
        reference_summaries (list): Ground-truth reference summaries.

    Returns:
        dict: Combined evaluation metrics.
    """
    query_embeddings = [embedding_model.encode(f"{tc['make']} {tc['model']} {tc['year']} {tc['issue']}") for tc in test_cases]
    doc_embeddings = embedding_model.encode(data['Document'].tolist())  # Dataset embeddings



    # Generate Summaries
    generated_summaries = []
    for tc in test_cases:
        result = rag_agent_with_relevance_check(tc)
        generated_summaries.append(result['summary'])

    # Summarization Evaluation
    summarization_metrics = evaluate_summarization(generated_summaries, reference_summaries)

    # Combine Results
    return { "summarization": summarization_metrics}


In [22]:
# Example test cases
test_cases = [
    {"make": "ford", "model": "escape", "year": 2005, "issue": "brake failure risk"},
    {"make": "toyota", "model": "camry", "year": 2012, "issue": "seatbelt defect"}
]

# Ground-truth relevance for each query (binary 1/0 for relevant/non-relevant documents)
true_relevance = [
    [1, 1, 0, 0, 0],  # Example: First two documents are relevant for query 1
    [1, 0, 0, 1, 0]   # Example: First and fourth documents are relevant for query 2
]

# Reference summaries for evaluation
reference_summaries = [
    "Brake failure caused by overheating. Fix includes replacing brake lines.",
    "Seatbelt defect causing failure to lock. Resolution involves replacing the seatbelt system."
]


In [23]:
results = evaluate_pipeline(test_cases, true_relevance, reference_summaries)

print("\nSummarization Evaluation:")
print(f"ROUGE-1: {results['summarization']['rouge-1']['f']:.2f}")
print(f"ROUGE-2: {results['summarization']['rouge-2']['f']:.2f}")
print(f"ROUGE-L: {results['summarization']['rouge-l']['f']:.2f}")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Summarization Evaluation:
ROUGE-1: 0.05
ROUGE-2: 0.00
ROUGE-L: 0.04
